# MobileNet

In [10]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
mobile = keras.applications.mobilenet.MobileNet()

In [ ]:
def prepare_image(file):
    img_path = 'MobileNet-inference-images/'
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
from IPython.display import Image
Image(filename='MobileNet-inference-images/1.PNG', width=300,height=200) 

In [ ]:
preprocessed_image = prepare_image('1.PNG')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

In [ ]:
Image(filename='MobileNet-inference-images/2.PNG', width=300,height=200) 

In [ ]:
preprocessed_image = prepare_image('2.PNG')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

In [ ]:
Image(filename='MobileNet-inference-images/3.PNG', width=300,height=200) 

In [ ]:
preprocessed_image = prepare_image('3.PNG')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

## Fine-tune MobileNet on cats and dogs

### Image preparation

In [ ]:
train_path = 'cats-and-dogs/train'
valid_path = 'cats-and-dogs/valid'
test_path = 'cats-and-dogs/test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=10, shuffle=False)

### Modify Model

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
x = mobile.layers[-6].output
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers[:-5]:
    layer.trainable = False

### Train the model

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=4, 
                    validation_data=valid_batches, validation_steps=2, epochs=30, verbose=2)

### Predict cats and dogs

In [ ]:
test_labels = test_batches.classes

In [ ]:
test_labels

In [ ]:
test_batches.class_indices

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

## Fine-tune MobileNet on sign language digits

### Image preparation

Dataset:
https://github.com/ardamavi/Sign-Language-Digits-Dataset
- 10 classes (digits 0-9)
- Class data:

  - class 0: 205 images
  - class 1: 206 images
  - class 2: 206 images
  - class 3: 206 images
  - class 4: 207 images
  - class 5: 207 images
  - class 6: 207 images
  - class 7: 206 images
  - class 8: 208 images
  - class 9: 204 images

In [18]:
!ls

Dataset  Examples  LICENSE  README.md  test  train  valid


In [19]:
rm -rf /home/ubuntu/custom/deeplizrd/Sign-Language-Digits-Dataset

In [ ]:
!git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset

In [1]:
# Save files to Sign Language Digits Dataset directory (which resides in current directory where this notebook is saved)
# Create train, valid, test folders in Sign Language Digits Dataset
%cd Sign-Language-Digits-Dataset/Dataset
%mkdir train
%mkdir valid
%mkdir test

/home/ubuntu/custom/deeplizrd/Sign-Language-Digits-Dataset/Dataset


In [3]:
# Move all class directories with images into train/
%mv 0/ 1/ 2/ 3/ 4/ 5/ 6/ 7/ 8/ 9/ train/

In [4]:
# Make class directories for valid and test data sets
%cd valid
%mkdir 0/ 1/ 2/ 3/ 4/ 5/ 6/ 7/ 8/ 9/

%cd ../test
%mkdir 0/ 1/ 2/ 3/ 4/ 5/ 6/ 7/ 8/ 9/

/home/ubuntu/custom/deeplizrd/Sign-Language-Digits-Dataset/Dataset/valid
/home/ubuntu/custom/deeplizrd/Sign-Language-Digits-Dataset/Dataset/test


In [5]:
# In the next cell:
# Using bash, loop through each class directory in /train and randomly move 30 images from each class into the 
# corresponding class directory in /valid and 5 images from each class into the corresponding class directory in /test.

In [17]:
!ls Dataset/train

0  1  2  3  4  5  6  7	8  9


In [12]:
%%bash
cd ../train
for ((i=0; i<=9; i++)); do
    a=$(find $i/ -type f | shuf -n 30)
    mv $a ../valid/$i/
    b=$(find $i/ -type f | shuf -n 5)
    mv $b ../test/$i/
done

bash: line 1: cd: ../train: No such file or directory
find: ‘0/’: No such file or directory
mv: missing destination file operand after '../valid/0/'
Try 'mv --help' for more information.
find: ‘0/’: No such file or directory
mv: missing destination file operand after '../test/0/'
Try 'mv --help' for more information.
find: ‘1/’: No such file or directory
mv: missing destination file operand after '../valid/1/'
Try 'mv --help' for more information.
find: ‘1/’: No such file or directory
mv: missing destination file operand after '../test/1/'
Try 'mv --help' for more information.
find: ‘2/’: No such file or directory
mv: missing destination file operand after '../valid/2/'
Try 'mv --help' for more information.
find: ‘2/’: No such file or directory
mv: missing destination file operand after '../test/2/'
Try 'mv --help' for more information.
find: ‘3/’: No such file or directory
mv: missing destination file operand after '../valid/3/'
Try 'mv --help' for more information.
find: ‘3/’: No suc

In [7]:
# Move back into current notebook directory
%cd ../..

/home/ubuntu/custom/deeplizrd/Sign-Language-Digits-Dataset


In [8]:
train_path = 'Sign-Language-Digits-Dataset/train'
valid_path = 'Sign-Language-Digits-Dataset/valid'
test_path = 'Sign-Language-Digits-Dataset/test'

In [9]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=10, shuffle=False)

NameError: name 'ImageDataGenerator' is not defined

### Modify Model

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
x = mobile.layers[-6].output
predictions = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers[:-23]:
    layer.trainable = False

### Train the model

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=18, 
                    validation_data=valid_batches, validation_steps=3, epochs=60, verbose=2)

### Predict sign language digits

In [ ]:
test_labels = test_batches.classes

In [ ]:
predictions = model.predict_generator(test_batches, steps=5, verbose=0)

In [ ]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['0','1','2','3','4','5','6','7','8','9']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')